### 1. 라이브러리 로드

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import os

from google.colab import drive
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# 시각화 설정
sns.set_theme(style="whitegrid")
plt.rcParams['font.family'] = 'sans-serif'

drive.mount('/content/drive')

base_path = '/content/drive/MyDrive/f1_project/data/'
model_path = os.path.join(base_path, 'f1_excitement_model.pkl')
scaler_path = os.path.join(base_path, 'scaler_features.pkl')

test_file_name = 'finaltest_data.csv'
test_file_path = os.path.join(base_path, test_file_name)
print(f"Loading Test Data from: {test_file_path}")

try:
    df = pd.read_csv(test_file_path)
    print(f"Data Loaded Successfully: {df.shape}")
    display(df.head())
except FileNotFoundError:
    print(f"\n[Error] 파일을 찾을 수 없습니다! 경로를 확인하세요: {file_path}")
    raise  # 에러 발생 시 중단

### 모델 로드, 데이터 전처리

In [ ]:
# 2. 모델, 스케일러, 데이터 로드
try:
    model = joblib.load(model_path)
    scaler = joblib.load(scaler_path)
    print("✅ Model & Scaler loaded successfully.")

    df_test = pd.read_csv(test_file_path)
    print(f"✅ Data loaded. Shape: {df_test.shape}")
except FileNotFoundError as e:
    print(f"❌ 파일을 찾을 수 없습니다: {e}")
    print("training.ipynb를 먼저 실행했는지, csv 파일명이 맞는지 확인하세요.")
    raise

# 3. Target Engineering (MyScore -> Excitement_Score 변환)
# 10점 만점을 모델 기준인 0~1 스케일로 변환합니다.
if 'MyScore' in df_test.columns:
    print("✅ 'MyScore' column found. Converting to normalized Excitement_Score...")
    df_test['Excitement_Score'] = df_test['MyScore'] / 10.0
elif 'Excitement_Score' in df_test.columns:
    print("✅ 'Excitement_Score' already exists.")
else:
    raise ValueError("CSV 파일에 'MyScore' 컬럼이 없습니다! 직접 매긴 점수 컬럼이 필요합니다.")

# 결측치 제거 (점수를 매기지 않은 미래 경기는 평가 제외)
initial_len = len(df_test)
df_test = df_test.dropna(subset=['Excitement_Score'])
print(f" -> Dropped NaN Targets: {initial_len} -> {len(df_test)} rows remaining.")

### 평가 과정

In [ ]:
# 4. 예측 실행
features = ['Chaos_Score', 'Lead_Changes', 'Gap_Std_Dev', 'Position_Gains_Total']

# Feature 컬럼 존재 여부 체크
if not all(col in df_test.columns for col in features):
    missing = [col for col in features if col not in df_test.columns]
    raise ValueError(f"테스트 파일에 다음 Feature 컬럼이 누락되었습니다: {missing}")

X_test = df_test[features]
y_true = df_test['Excitement_Score']

# 저장된 스케일러로 Transform
X_test_scaled = scaler.transform(X_test)
y_pred = model.predict(X_test_scaled)

# =============================================================================
# 5. 최종 성능 평가
# =============================================================================

mae = mean_absolute_error(y_true, y_pred)
r2 = r2_score(y_true, y_pred)
rmse = np.sqrt(mean_squared_error(y_true, y_pred)) # RMSE도 추가하면 더 전문적으로 보임

# [핵심 보완] 주요 성능 수치를 '표(Table)' 형태로 정리
metrics_df = pd.DataFrame({
    'Metric': ['MAE (Mean Absolute Error)', 'RMSE (Root Mean Squared Error)', 'R2 Score'],
    'Value': [mae, rmse, r2],
    'Interpretation': [
        f'평균 {mae*10:.2f}점 오차 (10점 만점 기준)',
        f'오차의 표준편차: {rmse*10:.2f}점',
        f'데이터 설명력: {r2*100:.1f}%'
    ]
})

print("\n" + "="*50)
print(f"🏁 Final Test Performance Summary (2025 Season)")
print("="*50)
display(metrics_df) # 표 형태로 깔끔하게 출력됨
print("="*50)

### 시각화

In [ ]:
# 6. 시각화: Actual vs Predicted Scatter Plot
plt.figure(figsize=(10, 6))

# 산점도 그리기
sns.scatterplot(x=y_true, y=y_pred, s=150, color='royalblue', alpha=0.8, label='Race Data')

# 기준선 (Perfect Prediction Line)
plt.plot([0, 1], [0, 1], 'r--', linewidth=2, label='Ideal Fit (y=x)')

# 각 점에 라운드 이름 달아주기 (어떤 경기를 잘 맞췄는지 확인용)
if 'GrandPrix' in df_test.columns:
    for i in range(len(df_test)):
        plt.text(y_true.iloc[i]+0.01, y_pred[i], df_test['GrandPrix'].iloc[i], fontsize=9, alpha=0.8)

plt.xlabel('My Manual Score (Normalized 0~1)', fontsize=12)
plt.ylabel('AI Model Prediction (0~1)', fontsize=12)
plt.title('2025 Final Evaluation: My Ratings vs AI Predictions', fontsize=14)
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()

result_df = df_test[['GrandPrix', 'MyScore', 'Excitement_Score']].copy()
result_df['AI_Pred_Norm'] = y_pred
result_df['AI_Pred_Score'] = y_pred * 10.0 # 10점 만점으로 환산
result_df['Diff'] = abs(result_df['MyScore'] - result_df['AI_Pred_Score'])

print("\n[Detailed Prediction Results]")
display(result_df.sort_values('Diff')) # 오차가 적은 순서대로 정렬